In [ ]:
%matplotlib inline

import sys
import datacube
import rasterio.crs
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from dea_tools.spatial import xr_rasterize
from dea_tools.plotting import rgb
from dea_tools.landcover import plot_land_cover

In [ ]:
dc = datacube.Datacube(app='rabbit_colours')

In [ ]:
#gdf = gpd.read_file('rabbit_sites_unique_1km.shp')
gdf = gpd.read_file('rabbit_sites_unique_10km.shp')

# Create a csv file to save results
#output = 'rabbit_sites_unique_1km.csv'
output = 'rabbit_sites_unique_10km.csv'
with open(output, 'w') as f:
    f.write('Id,Year,Red,Green,Blue\n')

# Loop through polygons
for index, row in gdf.iterrows():
    
    # Get site ID
    ID = str(row['id'])
    Location = str(row['location'])
    
    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    
    # Loop over years and load geomedian landsat data
    for y in range(2013, 2025):
        ls = dc.load(product='ga_ls8cls9c_gm_cyear_3',
                     measurements=['nbart_red', 'nbart_green', 'nbart_blue'],
                     time=(str(y), str(y)),
                     output_crs='EPSG:3577',
                     geopolygon=geom)
        
        lc = dc.load(product="ga_ls_landcover_class_cyear_3",
                     measurements=["level3", "level4"],
                     time=(str(y), str(y)),
                     output_crs="EPSG:3577",
                     geopolygon=geom)
        
        # Generate a polygon mask to keep only data within the polygon
        mask = xr_rasterize(gdf.iloc[[index]], ls)
        
        # Mask dataset to set pixels outside the polygon to `NaN`
        ls_site = ls.where(mask)

        # Add landcover classes to the mask
        ls_site = ls_site.where((lc.level4 !=  27) & # Woody Closed (> 65 %)
                                (lc.level4 !=  28) & # Woody Open (40 to 65 %)"
                                (lc.level4 !=  29) & # Woody Open (15 to 40 %)"
                                (lc.level3 != 124) & # Aquatic vegetation
                                (lc.level3 != 215) & # Artificial
                                (lc.level3 != 220))  # Water
        
        # Export data as tif
        #out_tif = 'subsets_incl_woody/%03d_%s_%i.tif'%(int(ID), Location.replace(' ', ''), y)
        #rgb = ls_site.isel(time=0).to_array()
        #write_cog(geo_im=rgb, fname=out_tif, overwrite=True)
        
        # Obtain mean and std dev of pixels
        stats = []
        for b in ['nbart_red', 'nbart_green', 'nbart_blue']:
            d = ls_site[b].values
            d = d[(d != -1) & (np.isnan(d) != 1)]
            if d.size > 1:
                stats.append('%.2f'%np.mean(d))
            else:
                stats.append('No valid pixels')
        
        # Write to csv
        
        with open(output, 'a') as f:
            f.write('%s,%s,%s\n'%(ID, y, ','.join(stats)))

        print(ID, y)
        